In [1]:
%load_ext autoreload
%autoreload 2

In [16]:
# assert len(hi_base_res) == len(hi_ivn_res) == len(lo_base_res) == len(lo_ivn_res) == len(low_ivn.keys) == len(high_ivn.keys)
# dupes = 0
# for lo_key, hi_key, hi_base, hi_ivn, lo_base, lo_ivn in \
#     zip(low_ivn.keys, high_ivn.keys, hi_base_res, hi_ivn_res, lo_base_res, lo_ivn_res):
#     key = (lo_key, hi_key)
#
#     # todo
#     if key in results:
#         dupes += 1
#
#     # base_eq =  hi_base == lo_base,
#     # ivn_eq= hi_ivn == lo_ivn,
#     low_base_eq_ivn = lo_base == lo_ivn,
#     high_base_eq_ivn = hi_base == hi_ivn
#
#     w = x = y = z = False
#     if high_base_eq_ivn:
#         if low_base_eq_ivn:
#             w = True        ##
#         else:
#             y = True
#     else:   # high changed
#         if low_base_eq_ivn:
#             x = True        ##
#         else:
#             z = True
#     results[key] = dict(
#         # key = key[1],
#         # high_base_res=hi_base,
#         # high_ivn_res=hi_ivn,
#         # low_base_res=lo_base,
#         # low_ivn_res=lo_ivn,
#
#         # base_eq =  hi_base == lo_base,
#         # ivn_eq= hi_ivn == lo_ivn,
#         # low_base_eq_ivn= lo_base == lo_ivn,
#         # high_base_eq_ivn= hi_base == hi_ivn
#
#         same_same=w,
#         diff_same=x,
#         same_diff=y,
#         diff_diff=z
#     )
#
# return dupes, len(hi_base_res)
#

from tests.interchange import test_abstraction_medium as tam
from antra import *
from antra.location import reduce_dim, location_to_str

from pprint import pp

In [ ]:
# tam.test_abstraction_medium()

In [11]:
e = tam.make_experiment()

[GraphInput({'leaf1': tensor(-1.), 'leaf2': tensor(-1.), 'leaf3': tensor(-1.)}),
 GraphInput({'leaf1': tensor(-1.), 'leaf2': tensor(-1.), 'leaf3': tensor(1.)}),
 GraphInput({'leaf1': tensor(-1.), 'leaf2': tensor(1.), 'leaf3': tensor(-1.)}),
 GraphInput({'leaf1': tensor(-1.), 'leaf2': tensor(1.), 'leaf3': tensor(1.)}),
 GraphInput({'leaf1': tensor(1.), 'leaf2': tensor(-1.), 'leaf3': tensor(-1.)}),
 GraphInput({'leaf1': tensor(1.), 'leaf2': tensor(-1.), 'leaf3': tensor(1.)}),
 GraphInput({'leaf1': tensor(1.), 'leaf2': tensor(1.), 'leaf3': tensor(-1.)}),
 GraphInput({'leaf1': tensor(1.), 'leaf2': tensor(1.), 'leaf3': tensor(1.)})]
[GraphInput({'leaf1': tensor(False), 'leaf2': tensor(False), 'leaf3': tensor(False)}),
 GraphInput({'leaf1': tensor(False), 'leaf2': tensor(False), 'leaf3': tensor(True)}),
 GraphInput({'leaf1': tensor(False), 'leaf2': tensor(True), 'leaf3': tensor(False)}),
 GraphInput({'leaf1': tensor(False), 'leaf2': tensor(True), 'leaf3': tensor(True)}),
 GraphInput({'leaf1'

In [12]:
find_abstr_res = e.find_abstractions()

In [18]:
success_list = []
res, mapping = find_abstr_res[0]
pp(res)
pp(mapping)

{(((('leaf1', 1.0), ('leaf2', 1.0), ('leaf3', 1.0)), (('hidden1[::,0]', 0.0),)), ((('leaf1', True), ('leaf2', True), ('leaf3', True)), (('node', True),))): tensor(False),
 (((('leaf1', 1.0), ('leaf2', 1.0), ('leaf3', 1.0)), (('hidden1[::,0]', 1.0),)), ((('leaf1', True), ('leaf2', True), ('leaf3', True)), (('node', True),))): tensor(True),
 (((('leaf1', 1.0), ('leaf2', 1.0), ('leaf3', 1.0)), (('hidden1[::,0]', 0.0),)), ((('leaf1', True), ('leaf2', True), ('leaf3', True)), (('node', False),))): tensor(True),
 (((('leaf1', 1.0), ('leaf2', 1.0), ('leaf3', -1.0)), (('hidden1[::,0]', 0.0),)), ((('leaf1', True), ('leaf2', True), ('leaf3', False)), (('node', True),))): tensor(True),
 (((('leaf1', 1.0), ('leaf2', 1.0), ('leaf3', -1.0)), (('hidden1[::,0]', 1.0),)), ((('leaf1', True), ('leaf2', True), ('leaf3', False)), (('node', True),))): tensor(False),
 (((('leaf1', 1.0), ('leaf2', 1.0), ('leaf3', -1.0)), (('hidden1[::,0]', 0.0),)), ((('leaf1', True), ('leaf2', True), ('leaf3', False)), (('nod

In [ ]:
for result, mapping in find_abstr_res[:1]:
    low_node = list(mapping["node"].keys())[0]
    low_loc = mapping["node"][low_node]
    red_low_loc = reduce_dim(low_loc, 0)
    if low_loc is None: continue
    print(f"Low node and loc: {low_node}{location_to_str(low_loc,add_brackets=True)}")
    print(f"Reduced loc: {red_low_loc}")

    success = True
    verify_mapping(experiment, mapping, result, low_inputs, low_model)
    # G, causal_edges = construct_graph(low_model,high_model, mapping, result, realizations_to_inputs, "node", "root")
    # cliques = find_cliques(G, causal_edges, 5)

    # print("cliques:", cliques)
    # x = input()

    for keys in result:
        # low_intervention, high_intervention = interventions
        low_ivn_key, high_ivn_key = keys
        low_ivn = experiment.low_keys_to_interventions[low_ivn_key]
        high_ivn = experiment.high_keys_to_interventions[high_ivn_key]

        _, low_res = low_model.intervene(low_ivn)
        _, high_res = high_model.intervene(high_ivn)

        x, y, z = tuple(low_ivn.base[f"leaf{i}"] for i in range(1,4))
        # print(f"{x=}")
        a = torch.stack([x, y, z], dim=-1)
        h = a.matmul(low_model.W) + low_model.B
        h = F.relu(h)

        assert torch.allclose(h, low_model.compute_node('hidden1', low_ivn.base))
        if low_node == "hidden1":
            ivn_value = low_ivn.intervention[low_node]
            h[red_low_loc] = ivn_value

        h2 = h.matmul(low_model.w) - low_model.b
        if low_node == "hidden2":
            ivn_value = low_ivn.intervention[low_node]
            h2 = ivn_value

        expected_low_res = h2 > 0

        n = high_ivn.base["leaf1"] & high_ivn.base["leaf2"]
        if "node" in high_ivn.intervention:
            n = high_ivn.intervention["node"]
        expected_high_res = n & high_ivn.base["leaf3"]

        assert expected_low_res == low_res
        assert expected_high_res == high_res
        assert (expected_low_res == expected_high_res) == result[keys]

        # print(f"{expected_low_res=}")
        # print(f"{expected_high_res=}")

        print("low_res", low_res)
        print("high_res", high_res)
        print("low intervention:",low_ivn.intervention.values)
        print(f"low loc {low_node}", location.location_to_str(low_ivn.location[low_node], add_brackets=True))
        print("lowbase:", low_ivn.base.values)
        print("high intervetion:", high_ivn.intervention.values)
        print("highbase:", high_ivn.base.values)
        print("success:", result[keys])
        #
        # print("\n\n")
        # verify_intervention(mapping,low_intervention, high_intervention, result[keys])

        if not result[keys]:
            success = False

    success_list.append((success,mapping))

pprint(success_list)

